# 카글 텍스트 분류 - 합성곱 신경망 활용 접근방법


- 이번 장은 앞서 00장에서 간략하게 설명하였던 합성곱 신경망을 활용하여 텍스트 분류 문제를 풀어보고자 한다. 합성곱 신경망은 주로 이미지에서 특징을 추출하여 이미지 판단을 하는 역할로 큰 성능을 이루었는데, 텍스트에서도 좋은 효과를 낼 수 있다는 점을 Yoon Kim (2014) 박사가 "Convolution Neural Network for Sentence Classificaion" (http://emnlp2014.org/papers/pdf/EMNLP2014181.pdf) 활용하여 입증하였다.

<img src="./Fig/fig1-cnn_text_classification.png"> 

**Fig1. Yoon Kim's Text Classification**

- RNN이 단어의 입력의 순서를 중요하게 반영한다면, CNN은 문장의 지역 정보를 보존하면서 각 문장 성분의 등장 정보들을 학습에 반영하는 구조로 풀어가고 있습니다. 학습을 하면서 각 필터 사이즈를 조절하면서 언어의 특징 값을 추출하게 되는데, 기존의 N-gram (2 그램, 3 그램) 방식과 유사하다고 볼 수 있습니다.

- 예를 들어 "나는 배가 고프다" 라는 문장을 2그램을 사용한다면, "나 는 / 는 배 / 배 가 / 가 고프 / 고프 다/" 로 각각 문장의 단어 성분을 쪼개어 활용 하는 접근방법을, 단어를 각 백터값을 투영하여 컨볼루션 필터값에 적용하는 원리입니다.

# 코드 설명

 - 이전 내용에서 기본적인 Kaggle 연동 및 데이터 분석 및 전처리를 진행하여, 여기에서는 관련된 주제 위주로 설명을 하겠습니다. Full Code는 http://Kaggle/BagOfWordsMeetsBagsOfPopcorn/cnn_text_classification-bagofwords-book_ver.ipynb 를 참조하시기 바랍니다. 

In [50]:
import os
from datetime import datetime

import os
import string
import tempfile
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

from datetime import datetime

# from tensorflow.python.keras.datasets import imdb
from tensorflow.python.keras.preprocessing.sequence import pad_sequences
from tensorflow.python.keras.preprocessing.text import Tokenizer
from tensorflow.python.keras.preprocessing import sequence

from tensorboard import summary as summary_lib

#전처리 lib
from bs4 import BeautifulSoup
import pandas as pd
import nltk
from nltk.corpus import stopwords

import re

In [51]:
import numpy as np
import json

FILE_DIR_PATH = './data/'
INPUT_TRAIN_DATA_FILE_NAME = 'input.npy'
LABEL_TRAIN_DATA_FILE_NAME = 'label.npy'
DATA_CONFIGS_FILE_NAME = 'data_configs.json'

input_data = np.load(open(FILE_DIR_PATH + INPUT_TRAIN_DATA_FILE_NAME, 'rb'))
label_data = np.load(open(FILE_DIR_PATH + LABEL_TRAIN_DATA_FILE_NAME, 'rb'))
prepro_configs = None

from sklearn.model_selection import train_test_split

TEST_SPLIT = 0.1
RNG_SEED = 13371447

input_train, input_eval, label_train, label_eval = train_test_split(input_data, label_data, test_size=TEST_SPLIT, random_state=RNG_SEED)

In [21]:
BATCH_SIZE = 16
NUM_EPOCHS = 10
vocab_size = 74065
embedding_size = 128

def mapping_fn(X, Y):
    input, label = {'text': X}, Y
    return input, label

def train_input_fn():
    dataset = tf.data.Dataset.from_tensor_slices((input_train, label_train))
    dataset = dataset.shuffle(buffer_size=len(input_train))
    dataset = dataset.batch(BATCH_SIZE)
    dataset = dataset.map(mapping_fn)
    dataset = dataset.repeat(count=NUM_EPOCHS)
    iterator = dataset.make_one_shot_iterator()
    
    return iterator.get_next()

def eval_input_fn():
    dataset = tf.data.Dataset.from_tensor_slices((input_eval, label_eval))
    dataset = dataset.shuffle(buffer_size=len(input_eval))
    dataset = dataset.batch(16)
    dataset = dataset.map(mapping_fn)
    iterator = dataset.make_one_shot_iterator()
    
    return iterator.get_next()

In [22]:
# x_len_train = np.array([min(len(x), sentence_size) for x in x_train_variable])
# x_len_test = np.array([min(len(x), sentence_size) for x in x_test_variable])

def model_fn(features, labels, mode, params):

    TRAIN = mode == tf.estimator.ModeKeys.TRAIN
    EVAL = mode == tf.estimator.ModeKeys.EVAL
    PREDICT = mode == tf.estimator.ModeKeys.PREDICT
    
    #embedding layer를 선언합니다.
    input_layer = tf.contrib.layers.embed_sequence(
                    features['text'],
                    vocab_size,
                    embedding_size,
                    initializer=params['embedding_initializer']
                    )
    # 현재 모델이 학습모드인지 여부를 확인하는 변수입니다.
    training = (mode == tf.estimator.ModeKeys.TRAIN)
    # embedding layer에 대한 output에 대해 dropout을 취합니다.
    dropout_emb = tf.layers.dropout(inputs=input_layer,
                                   rate=0.2,
                                   training=training)
    
    conv = tf.layers.conv1d(
            inputs=dropout_emb,
            filters=32,
            kernel_size=3,
            padding='same',
            activation=tf.nn.relu)
    
    pool = tf.reduce_max(input_tensor=conv, axis=1)
    hidden = tf.layers.dense(inputs=pool, units=250, activation=tf.nn.relu)
    dropout_hidden = tf.layers.dropout(inputs=hidden, rate=0.2, training=training)
#     logits = tf.layers.dense(inputs=dropout_hidden, units=1, name='result')
    logits = tf.layers.dense(inputs=dropout_hidden, units=1)
    
    #prediction 진행 시, None
    if labels is not None:
        labels = tf.reshape(labels, [-1, 1])

    if TRAIN:
        global_step = tf.train.get_global_step()
        loss = tf.losses.sigmoid_cross_entropy(labels, logits)
        train_op = tf.train.AdamOptimizer(0.001).minimize(loss, global_step)

        return tf.estimator.EstimatorSpec(mode=mode, train_op=train_op, loss = loss)
    
    elif EVAL:
        loss = tf.losses.sigmoid_cross_entropy(labels, logits)
        pred = tf.nn.sigmoid(logits)
        accuracy = tf.metrics.accuracy(labels, tf.round(pred))
        return tf.estimator.EstimatorSpec(mode=mode, loss=loss, eval_metric_ops={'acc': accuracy})
        
    elif PREDICT:
        return tf.estimator.EstimatorSpec(
            mode=mode,
            predictions={
                'prob': tf.nn.sigmoid(logits),
            }
        )

In [23]:
params = {'embedding_initializer': tf.random_uniform_initializer(-1.0, 1.0)}

model_dir = os.path.join(os.getcwd(), "checkpoint/cnn_model")
os.makedirs(model_dir, exist_ok=True)

config_tf = tf.estimator.RunConfig()
config_tf._save_checkpoints_steps = 100
config_tf._save_checkpoints_secs = None
config_tf._keep_checkpoint_max =  2
config_tf._log_step_count_steps = 100

est = tf.estimator.Estimator(model_fn, model_dir=model_dir, config=config_tf, params=params)

INFO:tensorflow:Using config: {'_model_dir': '/Users/user/git/DeepNLP/Kaggle/BagOfWordsMeetsBagsOfPopcorn/checkpoint/cnn_model', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': 100, '_save_checkpoints_secs': None, '_session_config': None, '_keep_checkpoint_max': 2, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x114890400>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [40]:
tf.logging.set_verbosity(tf.logging.INFO)
print(tf.__version__)
time_start = datetime.utcnow()
print("Experiment started at {}".format(time_start.strftime("%H:%M:%S")))
print(".......................................") 

est.train(train_input_fn, steps=10)
est.evaluate(eval_input_fn)

time_end = datetime.utcnow()
print(".......................................")
print("Experiment finished at {}".format(time_end.strftime("%H:%M:%S")))
print("")
time_elapsed = time_end - time_start
print("Experiment elapsed time: {} seconds".format(time_elapsed.total_seconds()))

predictions = np.array([p['prob'][0] for p in est.predict(input_fn=eval_input_fn)])

1.8.0
Experiment started at 02:19:54
.......................................
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /Users/user/git/DeepNLP/Kaggle/BagOfWordsMeetsBagsOfPopcorn/checkpoint/cnn_model/model.ckpt-221
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 222 into /Users/user/git/DeepNLP/Kaggle/BagOfWordsMeetsBagsOfPopcorn/checkpoint/cnn_model/model.ckpt.
INFO:tensorflow:loss = 0.6553388, step = 222
INFO:tensorflow:Saving checkpoints for 231 into /Users/user/git/DeepNLP/Kaggle/BagOfWordsMeetsBagsOfPopcorn/checkpoint/cnn_model/model.ckpt.
INFO:tensorflow:Loss for final step: 0.6698082.
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2018-08-26-02:20:00
INFO:tensorflow:Graph was finalized.
INFO

# Test Dataset 불러오기

In [53]:
import pandas as pd

default_path = '/Users/user/.kaggle/competitions/word2vec-nlp-tutorial/'

In [60]:
#매번 위와 같은 작업을 반복할 수 없으니 함수로 만들어봅니다.
def review_to_words( raw_review ):
    # 1. HTML 태그 지우기
    review_text = BeautifulSoup(raw_review, "html5lib").get_text()
    
    # 2. 영어가 아닌 특수문자들을 공백(" ")으로 바꾸기
    letters_only = re.sub("[^a-zA-Z]", " ", review_text)

    # 3. 대문자들을 소문자로 바꾸고 공백단위로 텍스트들을 나누기
    words = letters_only.lower().split()

    # 4. 파이썬에서 리스트보다 set에서의 찾기가 더 빠르다고 합니다.
    stops = set(stopwords.words("english"))
    
    # 5. 불용어가 아닌 것만 남기기
    meaningful_words = [w for w in words if not w in stops]
    
    # 6. 불용어가 아닌 텍스트를 공백 단위로 나누어서 리턴
    return( " ".join( meaningful_words ))

In [61]:
#테스트 데이터 로드
test = pd.read_csv(default_path+"testData.tsv", header=0, delimiter="\t", quoting=3 )

print ("test dataset shape: {}".format(test.shape))

# 불용어 제거 및 태그를 삭제 후, 데이터를 저장할 장소를 만들자
num_reviews = len(test["review"])
clean_test_reviews = []

print ("테스트 영화 리뷰 전처리 진행...\n")
for i in range(0,num_reviews):
    if( (i+1) % 1000 == 0 ):
        print ("Review %d of %d\n" % (i+1, num_reviews))
    clean_review = review_to_words( test["review"][i] )
    clean_test_reviews.append( clean_review )

test dataset shape: (25000, 2)
테스트 영화 리뷰 전처리 진행...

Review 1000 of 25000

Review 2000 of 25000

Review 3000 of 25000

Review 4000 of 25000

Review 5000 of 25000

Review 6000 of 25000

Review 7000 of 25000

Review 8000 of 25000

Review 9000 of 25000

Review 10000 of 25000

Review 11000 of 25000

Review 12000 of 25000

Review 13000 of 25000

Review 14000 of 25000

Review 15000 of 25000

Review 16000 of 25000

Review 17000 of 25000

Review 18000 of 25000

Review 19000 of 25000

Review 20000 of 25000

Review 21000 of 25000

Review 22000 of 25000

Review 23000 of 25000

Review 24000 of 25000

Review 25000 of 25000



In [58]:
#테스트 파일은 이렇게 생겼다고 합니다.
print (test.head())

#이 파일은 "sentiment" 행이 없습니다.

           id                                             review
0  "12311_10"  "Naturally in a film who's main themes are of ...
1    "8348_2"  "This movie is a disaster within a disaster fi...
2    "5828_4"  "All in all, this is a movie for kids. We saw ...
3    "7186_2"  "Afraid of the Dark left me with the impressio...
4   "12128_7"  "A very accurate depiction of small time mob l...


In [59]:
# 예측된 모델을 불러 체크포인트로 결과치를 불러온다.

sentimental = []

for i in range(len(clean_test_reviews)):
    if ( (i+1) % 1000 == 0):
        print ("Current Progress %d \n" % (i+1))
    sentimental.append(print_predictions([clean_test_reviews[i]]))
    
#알아보기 쉽게 데이터랑 붙여두는 편이 좋을 거 같습니다.
output = pd.DataFrame( data={"id":test["id"], "sentiment":sentimental} )

#지금까지 처리한 결과를 파일로 저장합니다.
output.to_csv( "Bag_of_Words_model_test.csv", index=False, quoting=3 )

NameError: name 'word_index' is not defined

## 이전 코드

In [ ]:
#input function

#각 문장의 길이를 계산한다, max 길이는 200

x_len_train = np.array([min(len(x), sentence_size) for x in x_train_variable])
x_len_test = np.array([min(len(x), sentence_size) for x in x_test_variable])

def parser(x, length, y):
    features = {"x": x, "len": length}
    return features, y

#len을 활용하여 기존 전처리 이후의 길이를 보존
#from_tensor_slices를 활용하면 numpy 데이터 구조에서 쉽게 변환

def train_input_fn():
    dataset = tf.data.Dataset.from_tensor_slices((x_train, x_len_train, y_train))
    dataset = dataset.shuffle(buffer_size=len(x_train_variable))
    dataset = dataset.batch(100)
    dataset = dataset.map(parser)
    dataset = dataset.repeat()
    iterator = dataset.make_one_shot_iterator()
    
    return iterator.get_next()

def eval_input_fn():
    dataset = tf.data.Dataset.from_tensor_slices((x_test, x_len_test, y_test))
    dataset = dataset.batch(100)
    dataset = dataset.map(parser)
    iterator = dataset.make_one_shot_iterator()
    
    return iterator.get_next()

# CNN Classification

CNN을 활용하여 text를 분류해보자, n-gram의 효과로 활용

https://www.semanticscholar.org/paper/Learning-to-Rank-Short-Text-Pairs-with-Deep-Neural-Severyn-Moschitti/452f7411af7d471dd3ba84c2b06b2aaffc38cdb9

Embedding Layer -> Dropout -> Conv1D -> GlobalMax1D -> Hidden Dense Layer -> Dropout -> Output Layer

In [ ]:
all_classifiers = {}

def train_and_evaluate(classifier):
    # 예측 테스트를 위해 모델을 학습시키고 저장한다.
    all_classifiers[classifier.model_dir] = classifier
    classifier.train(input_fn=train_input_fn, steps=1)
    eval_results = classifier.evaluate(input_fn=eval_input_fn)
    predictions = np.array([p['logistic'][0] for p in classifier.predict(input_fn=eval_input_fn)])
    
    # name scopes의 재사용을 위해 graph를 reset한다.
    tf.reset_default_graph()
    
    pr = summary_lib.pr_curve('precision_recall', predictions=predictions, labels=y_test.astype(bool),
                             num_thresholds=21)
    
    with tf.Session() as sess:
        writer = tf.summary.FileWriter(os.path.join(classifier.model_dir, 'eval'), sess.graph)
        writer.add_summary(sess.run(pr), global_step=0)
        writer.close()

In [ ]:
#head: pre-made estimator로 평가를 할 때, 일정한 함수를 사용하게 세팅
head = tf.contrib.estimator.binary_classification_head()

def cnn_model_fn(features, labels, mode, params):
    #embedding layer를 선언한다.
    input_layer = tf.contrib.layers.embed_sequence(
                    features['x'],
                    vocab_size,
                    embedding_size,
                    initializer=params['embedding_initializer']
                    )

    training = (mode == tf.estimator.ModeKeys.TRAIN)
    dropout_emb = tf.layers.dropout(inputs=input_layer,
                                   rate=0.2,
                                   training=training)

    conv = tf.layers.conv1d(
            inputs=dropout_emb,
            filters=32,
            kernel_size=3,
            padding='same',
            activation=tf.nn.relu)
    
    pool = tf.reduce_max(input_tensor=conv, axis=1)
    hidden = tf.layers.dense(inputs=pool, units=250, activation=tf.nn.relu)  
    dropout_hidden = tf.layers.dropout(inputs=hidden, rate=0.2, training=training)
    logits = tf.layers.dense(inputs=dropout_hidden, units=1)
    
    #prediction 진행 시, None
    if labels is not None:
        labels = tf.reshape(labels, [-1, 1])
    
    optimizer = tf.train.AdamOptimizer() #여러가지 Optimizer 활용가능
    
    def _train_op_fn(loss):
#         tf.summary('loss', loss)
        return optimizer.minimize(
                loss=loss,
                global_step=tf.train.get_global_step())

    
    return head.create_estimator_spec(
        features=features,
        labels=labels,
        mode=mode,
        logits=logits,
        train_op_fn=_train_op_fn)


cnn_classifier = tf.estimator.Estimator(model_fn=cnn_model_fn,
                                        model_dir=os.path.join(model_dir, 'cnn'),
                                        params=params)

In [ ]:
#학습 후, 결과치를 tensorboard로 확인
# tensorboard --logdir=./checkpoint/cnn_classifier/
train_and_evaluate(cnn_classifier)

In [ ]:
# 직접 prediction으로 테스트 해 본다

def text_to_index(sentence):
    # Remove punctuation characters except for the apostrophe
    translator = str.maketrans('', '', string.punctuation.replace("'", ''))
    tokens = sentence.translate(translator).lower().split()
    return np.array([1] + [word_index[t] if t in word_index else oov_id for t in tokens])

def print_predictions(sentences):
    indexes = [text_to_index(sentence) for sentence in sentences]
    x = sequence.pad_sequences(indexes, 
                               maxlen=sentence_size, 
                               truncating='post',
                               padding='post',
                               value=pad_id)
    length = np.array([min(len(x), sentence_size) for x in indexes])
    predict_input_fn = tf.estimator.inputs.numpy_input_fn(x={"x": x, "len": length}, shuffle=False)
    predictions = {}
    for path, classifier in all_classifiers.items():
        predictions[path] = [p['logistic'][0] for p in classifier.predict(input_fn=predict_input_fn)]
    for idx, sentence in enumerate(sentences):
        print(sentence)
        for path in all_classifiers:
            print("\t{} {}".format(path, predictions[path][idx]))
#             predictions[path][idx]
    
    return predictions[path][idx]

In [ ]:
print_predictions([
    'I do not like this movie'
])

In [ ]:
print_predictions(['fuck you', 'this movie sucks'])

지금까지 했던 것을 모두 활용하여 제출용 데이터를 만들어봅시다.

In [ ]:
import pandas as pd

default_path = '/Users/user/.kaggle/competitions/word2vec-nlp-tutorial/'

In [ ]:
#테스트 데이터 로드
test = pd.read_csv(default_path+"testData.tsv", header=0, delimiter="\t", quoting=3 )

print ("test dataset shape: {}".format(test.shape))

# 불용어 제거 및 태그를 삭제 후, 데이터를 저장할 장소를 만들자
num_reviews = len(test["review"])
clean_test_reviews = []

print ("테스트 영화 리뷰 전처리 진행...\n")
for i in range(0,num_reviews):
    if( (i+1) % 1000 == 0 ):
        print ("Review %d of %d\n" % (i+1, num_reviews))
    clean_review = review_to_words( test["review"][i] )
    clean_test_reviews.append( clean_review )

In [ ]:
#테스트 파일은 이렇게 생겼다고 합니다.
print (test.head())

#이 파일은 "sentiment" 행이 없습니다.

In [ ]:
# 예측된 모델을 불러 체크포인트로 결과치를 불러온다.

sentimental = []

for i in range(len(clean_test_reviews)):
    if ( (i+1) % 1000 == 0):
        print ("Current Progress %d \n" % (i+1))
    sentimental.append(print_predictions([clean_test_reviews[i]]))
    
#알아보기 쉽게 데이터랑 붙여두는 편이 좋을 거 같습니다.
output = pd.DataFrame( data={"id":test["id"], "sentiment":sentimental} )

#지금까지 처리한 결과를 파일로 저장합니다.
output.to_csv( "Bag_of_Words_model_test.csv", index=False, quoting=3 )

In [ ]:
#알아보기 쉽게 데이터랑 붙여두는 편이 좋을 거 같습니다.
output = pd.DataFrame( data={"id":test["id"], "sentiment":final_result} )

#결과값 저장
output.to_csv( "final_bof.csv", index=False, quoting=3 )'

#0.5 기준으로 값들을 변환

def correct_val(x):
    if x >= 0.5:
        x = 1
    else:
        x = 0
    
    return x

final_result = output['sentiment'].apply(correct_val)